In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pya
import klayout.db as pya

In [2]:
layout = pya.Layout()
unit_trans = pya.Trans()
dbu = 0.005
zero = pya.DPoint(0, 0)

# Set the layout database unit (DBU) in units of micrometers
# By default it is 0.001 (1 nm) which will work for low resolution writes on the EBL
layout.dbu = 0.005 # could change to 0.000125 for Mode 5

#layer and datatype
metal5 = layout.layer(72,20)
metal4 = layout.layer(71,20)
metal3 = layout.layer(70,20)
metal2 = layout.layer(69,20)
metal1 = layout.layer(68,20)
via4 = layout.layer(71, 44)
via3 = layout.layer(70, 44)
via2 = layout.layer(69, 44)
via1 = layout.layer(68, 44)
li1 = layout.layer(67,20)
licon = layout.layer(66,44)
psdm = layout.layer(94, 20)
mcon = layout.layer(67, 44)
diffusion = layout.layer(65, 20)
psdm = layout.layer(94, 20)

#omitted layers generated differently
layers = [psdm, metal1, metal2, metal3, metal4, metal5]
#Cell name and instantiation
Inductor = layout.create_cell('3nH_guard')


In [3]:
#function converts lists of tuples into pya Dpoint objects
def make_points(points):
    for i in range(len(points)):
        points[i] = pya.DPoint(points[i][0], points[i][1])
    return points

def make_Ipoints(points):
    for i in range(len(points)):
        points[i] = pya.Point(points[i][0], points[i][1])
    return points

#to send out of dbu in order to perform operations limited by integers
to_dbu = pya.CplxTrans(dbu)
from_dbu = to_dbu.inverted()

## Paramaters

In [4]:
#Radius from center to outer corner
Outer_Rad = 250
#Width of Metal
Metal_Width = 20
#Spacing between metal loops
Metal_Space = 15
#Metal loops 
Turns = 2

#potentially 3 nH: 250/20/15/2

## Inductor Geometry

In [5]:
#This section will generate one trapezoid of the outer ring, later sections will allow iterations

#diagonal of the metal trapezoid
Delta_R = Metal_Width / (np.cos(np.pi/8))
#Diagonal space between turns
Delta_Turn = Metal_Space / (np.cos(np.pi/8))
#Diagonal value of both spacing and metal
Delta_Net = Delta_R + Delta_Turn

#Radius in corner after each turn
New_Rad = Outer_Rad-Delta_R

#flat side "radius"
flat_rad = Outer_Rad*np.cos(np.pi/8)

#Step-down params

# height of box surrounding ladder
ladder_height = 2*Metal_Width+Metal_Space
#end of ladder flattened to spiral
flat_end_width = np.sqrt(2)*Metal_Width
#remaining distance to end ladder "box"
flat_end_slack = (ladder_height) / np.tan(np.pi/4) #my dumbass forgot to put tan45 but it didn't matter bc it was 1 anyways
#slack and flattended bit combined
ladder_width = flat_end_width + flat_end_slack
#distance towards center ladder moves each ring
ladder_space = Metal_Width+Metal_Space
#offset to maintain metal spacing between diagonals
# offset = ( Metal_Space / (2*np.sqrt(2)) ) - (ladder_width/2 - flat_end_width)


#-ladder_width/2


for i in range(Turns):
    Rad = Outer_Rad-(Delta_Net*i)
    I_Rad = New_Rad-(Delta_Net*i)
    I_Rad_Next = New_Rad-(Delta_Net*(i+1))
    polygon_points = [(-(Rad)*np.sin(np.pi/8), Rad*np.cos(np.pi/8)),
                      (Rad*np.sin(np.pi/8), Rad*np.cos(np.pi/8)),
                      (I_Rad*np.sin(np.pi/8), I_Rad*np.cos(np.pi/8)),
                      (-I_Rad*np.sin(np.pi/8), I_Rad*np.cos(np.pi/8))]
    make_points(polygon_points)
    
    offset = (I_Rad_Next*np.sin(np.pi/8) - ladder_width/2)
    
    if i < (Turns-1):
        # - (i*Metal_Space*np.sqrt(2))
        ladder_points = [(ladder_width/2-flat_end_width, flat_rad-(ladder_space*(i))),
                     (ladder_width/2, flat_rad-(ladder_space*(i))),
                     (-ladder_width/2+flat_end_width, flat_rad-ladder_height-(ladder_space*(i))),
                     (-ladder_width/2, flat_rad-ladder_height-(ladder_space*(i)))]
        make_points(ladder_points)
        ladder = pya.DPolygon(ladder_points).transformed(pya.DTrans(-offset+10, 0))
        Inductor.shapes(metal5).insert(ladder)
    
    #generate subtraction shape
#     if i == 0:
#         sub_offset = 0
#     else:
    sub_offset = flat_end_width+Metal_Space*np.sqrt(2)
    subtract_points = [(ladder_width/2, flat_rad-(ladder_space*(i))),
                       (ladder_width/2+Metal_Space*np.sqrt(2), flat_rad-(ladder_space*(i))),
                       (ladder_width/2+Metal_Space*(np.sqrt(2))-Metal_Width, flat_rad-(ladder_space*(i))-Metal_Width),
                       (ladder_width/2-Metal_Width, flat_rad-(ladder_space*(i))-Metal_Width)]
    make_points(subtract_points)

    for j in range(8):
        if j == 0:
            #Trap and region def
            trap_polygon = pya.DPolygon(polygon_points).transform(pya.CplxTrans(1, 45*j, 0, 0, 0))
            trap = pya.Region()
            #insert into region (from_dbu to conver to standard Polygon)
            trap.insert(from_dbu * trap_polygon)
            #repeat for subtraction
            trap_sub = pya.Region() 
            trap_sub_polygon = pya.DPolygon(subtract_points).transformed(pya.DTrans(-offset+10-sub_offset, 0))
            trap_sub.insert(from_dbu * trap_sub_polygon)
            cut = trap-trap_sub
            for polygon in cut.each():
                #return to Dpoly in dbu
                cut_dpoly = to_dbu*polygon
                Inductor.shapes(metal5).insert(cut_dpoly)
        else:
            trap = pya.DPolygon(polygon_points).transform(pya.CplxTrans(1, 45*j, 0, 0, 0))
            Inductor.shapes(metal5).insert(trap)

            

In [6]:
layout.write('Just Metal.gds')

## Guard Ring

In [7]:
sqr_con = 0.17
if sqr_con**2 < .0561 :
    sqr_li = 0.24
else:
    sqr_li = sqr_con
sqr_sep = 0.2

Guard_Dist = 35
r45 = pya.DCplxTrans(1, 45, 0, 0.0,0.0)


#width of m1/diffusion region prior to via ring. MCON/LICON Width 0.17, DIST BTWN 0.19, Fit 3 connections within width
#sqrt 0.12 for diagonal regions
Sub_Guard_Width = (sqr_con*np.sqrt(2)*3) + (sqr_sep*4)
Delta_Sub_Guard_R = Sub_Guard_Width / (np.cos(np.pi/8))
Sub_Guard_Rad = Guard_Dist+Outer_Rad
Sub_Guard_I_Rad = Sub_Guard_Rad - Delta_Sub_Guard_R

#li1 strips on straights and diagonals
Straight_li1_length = 3*sqr_con+2*sqr_sep+0.16


#make trapezoids for diffusion/metal 1
Sub_Guard_Trap_Points = [(-(Sub_Guard_Rad)*np.sin(np.pi/8), Sub_Guard_Rad*np.cos(np.pi/8)),
                          (Sub_Guard_Rad*np.sin(np.pi/8), Sub_Guard_Rad*np.cos(np.pi/8)),
                          (Sub_Guard_I_Rad*np.sin(np.pi/8), Sub_Guard_I_Rad*np.cos(np.pi/8)),
                          (-Sub_Guard_I_Rad*np.sin(np.pi/8), Sub_Guard_I_Rad*np.cos(np.pi/8))]

Sub_Guard_Rect_Points = [(-(Sub_Guard_Rad)*np.sin(np.pi/8), Sub_Guard_Rad*np.cos(np.pi/8)),
                          (Sub_Guard_Rad*np.sin(np.pi/8), Sub_Guard_Rad*np.cos(np.pi/8)),
                          (Sub_Guard_Rad*np.sin(np.pi/8), Sub_Guard_I_Rad*np.cos(np.pi/8)),
                          (-Sub_Guard_Rad*np.sin(np.pi/8), Sub_Guard_I_Rad*np.cos(np.pi/8))]

make_points(Sub_Guard_Trap_Points)
make_points(Sub_Guard_Rect_Points)

#LI LAYER (NO 45 degrees, just make rectangles)    
# removes 0.19 from either side of the sub guard ring
# LI_Rad = Sub_Guard_Rad - (0.19/(np.cos(np.pi/8)))
# LI_I_Rad = Sub_Guard_I_Rad + (0.19/(np.cos(np.pi/8)))
# LI_Points = [(-(LI_I_Rad)*np.sin(np.pi/8), LI_Rad*np.cos(np.pi/8)),
#                             (LI_I_Rad*np.sin(np.pi/8), LI_Rad*np.cos(np.pi/8)),
#                             (LI_I_Rad*np.sin(np.pi/8), LI_I_Rad*np.cos(np.pi/8)),
#                             (-LI_I_Rad*np.sin(np.pi/8), LI_I_Rad*np.cos(np.pi/8))]
# make_points(LI_Points)
    
#metal1, 90 degree diffusion, psdm   
psdm_rad = Sub_Guard_Rad + 10 #TEMPORARY
psdm_oct_points = []


for i in range(8):
    psdm_oct_points.append(pya.DCplxTrans(1, 45*i, 0, 0.0,0.0)*pya.DVector(-(psdm_rad)*np.sin(np.pi/8), psdm_rad*np.cos(np.pi/8)))
    Inductor.shapes(metal1).insert(pya.DPolygon(Sub_Guard_Trap_Points).transform(pya.CplxTrans(1, 45*i, 0, 0, 0)))
    if i%2 == 0:
        Inductor.shapes(diffusion).insert(pya.DPolygon(Sub_Guard_Rect_Points).transform(pya.CplxTrans(1, 45*i, 0, 0, 0)))
#     Inductor.shapes(li).insert(pya.DPolygon(LI_Points).transform(pya.CplxTrans(1, 45*i, 0, 0, 0)))
    
Inductor.shapes(psdm).insert(pya.DPolygon(psdm_oct_points))
    
#ORTHOGONAL CELLS - MCON & LICON (0, 90, 180, 270)

con_region_width = Sub_Guard_I_Rad*np.sin(np.pi/8)*2
N90 = int((con_region_width-sqr_sep*2) / (sqr_con+sqr_sep))

N45 = int((con_region_width-sqr_sep*2) / (sqr_li*np.sqrt(2)+sqr_sep))
# print(N90)
# print(N45)

straight = pya.Region()
diagonal = pya.Region()
diffusion_diagonal_reg= pya.Region()
straight_li1 = pya.Region()
sqr_li_reg = pya.Region()
sqr_regs = [straight, diagonal]

for i in range(2):
    if i == 0:
        N = N90
        delta_sqr = sqr_con/2
        corner_align = sqr_con/2
        from_corner_y = (Sub_Guard_Width - (3*(2*delta_sqr) + 2*(sqr_sep)))/2
        from_corner_x = (con_region_width - ((N)*(2*delta_sqr) + (N-1)*(sqr_sep)))/2


        #centers will be different to maintain distance from ring edges
        center_x = -Sub_Guard_I_Rad*np.sin(np.pi/8)+corner_align + from_corner_x
        center_y = Sub_Guard_I_Rad*np.cos(np.pi/8)+corner_align + from_corner_y


        delta_center = 2*delta_sqr+sqr_sep

        for j in range(3):
            for k in range(N): 
                box_vector = pya.DVector(center_x+k*delta_center,center_y+j*delta_center)
                sqr_place = pya.DBox(sqr_con).transformed(pya.DCplxTrans(1, 45*i, 0, center_x+k*delta_center,center_y+j*delta_center))
                sqr_regs[i].insert(from_dbu*sqr_place)
                if (j == 1):
                        li1_strip = pya.DBox(sqr_con, Straight_li1_length).transformed(pya.DTrans(box_vector))
                        straight_li1.insert(from_dbu*li1_strip)
                        
                        
    #lots are repeated due to changing pad generation method. 
    #Previous: Generate 45deg rotated squares and apply global rotations for alignment -note: other method now vector-may make better later
    #New: Generate Vectors pointing to the center of each square and rotating vectors before generating boxes.
    else:
        N = N45 
        delta_sqr = sqr_li*np.sqrt(2)/2
        corner_align = np.sqrt(2)*sqr_con/2
        
        from_corner_y = (Sub_Guard_Width - (3*(2*delta_sqr) + 2*(sqr_sep)))/2
        from_corner_x = (con_region_width - ((N)*(2*delta_sqr) + (N-1)*(sqr_sep)))/2

        #centers will be different to maintain distance from ring edges
        center_x = -Sub_Guard_I_Rad*np.sin(np.pi/8)+corner_align + from_corner_x
        center_y = Sub_Guard_I_Rad*np.cos(np.pi/8)+corner_align + from_corner_y
        
        delta_center = 2*delta_sqr+sqr_sep
                
        for j in range(3):
            for k in range(N): 
                box_vector = r45*pya.DVector(center_x+k*delta_center,center_y+j*delta_center)
                sqr_place = pya.DBox(sqr_con).transformed(pya.DTrans(box_vector))
                sqr_li_place = pya.DBox(sqr_con, sqr_con+0.16).transformed(pya.DTrans(box_vector))
                sqr_regs[i].insert(from_dbu*sqr_place)
                sqr_li_reg.insert(from_dbu*sqr_li_place)
                
                #throwing in vector centers for diffusion box placement (test)
                if (j == 1):
                    diffusion_size = Sub_Guard_Width
                    #diffusion_vector = r45*pya.DVector(center_x+k*delta_center,center_y+j*delta_center)
                    diff_sqr_place = pya.DBox(diffusion_size).transformed(pya.DTrans(box_vector))
                    diffusion_diagonal_reg.insert(from_dbu*diff_sqr_place)
                    
                
                
for i in range(4):
    straight_dup = straight.dup().transform(pya.CplxTrans(1, 90*i, 0, 0, 0))
    diag_dup = diagonal.dup().transform(pya.CplxTrans(1, 90*i, 0, 0, 0))  
    diff_diag_dup = diffusion_diagonal_reg.dup().transform(pya.CplxTrans(1, 90*i, 0, 0, 0))  
    straight_li1_dup = straight_li1.dup().transform(pya.CplxTrans(1, 90*i, 0, 0, 0))  
    sqr_li1_dup = sqr_li_reg.dup().transform(pya.CplxTrans(1, 90*i, 0, 0, 0))
    for polygon in straight_dup:
        Inductor.shapes(mcon).insert(to_dbu*polygon)
        Inductor.shapes(licon).insert(to_dbu*polygon)
    for polygon in diag_dup:
        Inductor.shapes(mcon).insert(to_dbu*polygon)
        Inductor.shapes(licon).insert(to_dbu*polygon)
    for polygon in diff_diag_dup:
        Inductor.shapes(diffusion).insert(to_dbu*polygon)
    for polygon in straight_li1_dup:
        Inductor.shapes(li1).insert(to_dbu*polygon)  
    for polygon in sqr_li1_dup:
        Inductor.shapes(li1).insert(to_dbu*polygon)  


## vias_gen enrique edition

In [8]:
#periphery rules for metals and vias (min size, spacing btwn, overlap of corresponding metal)

via_size = 3
via_start = 1
via_end = 5

v1_side = 0.15
v1v1 = 0.17
v1m1 = 0.055

v2_side = 0.2
v2v2 = 0.2
v2m2 = 0.04

v3_side = 0.2
v3v3 = 0.2
v3m3 = 0.06

v4_side = 0.8
v4v4 = 0.8
v4m4 = 0.19

metal_layers = [metal1, metal2, metal3, metal4, metal5]
via_layers = [via1, via2, via3, via4, via4]
via_sides = [v1_side, v2_side, v3_side, v4_side]
via_spacing = [v1v1, v2v2, v3v3, v4v4]
via_overlap = [v1m1, v2m2, v3m3, v4m4]

# parameters for m1 to substrate contacts
mcon_side = 0.17
mcon_space = 0.2
licon_side_side = 0.17
to_diff_order = [diffusion, mcon, licon, li1]

#

via_start_test = 1 
via_end_test = 5
#if you want to know the smalles via you can make with that layer range
via_min_size = via_sides[via_end_test-2] + 2 * via_overlap[via_end_test-2]
# print(via_min_size)

def gen_via(via_size, via_start, via_end):
    metals = []
    vias = []
    layer_order = []
    
    #i ranges from start layer to end layer 
    for i in range(via_start, via_end+1, 1):
        metals.append(pya.Region())
        metals[i-1].insert(from_dbu * pya.DBox(via_size))
        layer_order.append(metal_layers[i-1])
        
    #i ranges from start layer to end layer -1
    for i in range(via_start, via_end, 1):
        N = int( (via_size - 2*via_overlap[i-1]) / ((via_sides[i-1] + via_spacing[i-1]) ) )
        edge_gap = (-via_size/2) + (via_size - ( N * via_sides[i-1] + (N-1) * via_spacing[i-1] ) )/2 + via_sides[i-1]/2
        btwn_via = via_sides[i-1] + via_spacing[i-1]
        vias.append(pya.Region())
        for a in range(N):
            for b in range(N):
                displace = pya.DTrans(edge_gap+a*btwn_via, edge_gap+b*btwn_via) 
                vias[i-1].insert(from_dbu * pya.DBox(via_sides[i-1]).transformed(displace))
        layer_order.append(via_layers[i-1])
    return metals+vias, layer_order

# Cheap knock-off of gen_via for diffusion/local interconnects for the viaas
def gen_subvia(via_size):
    regions = []
    N = int( (via_size - 2*mcon_side) / ((mcon_side + mcon_space) ) )
    edge_gap = (-via_size/2) + (via_size - ( N * mcon_side + (N-1) * mcon_space ) )/2 + mcon_side/2
    btwn_via = mcon_side + mcon_space

    for i in range(len(to_diff_order)):
        regions.append(pya.Region())
        if i == 0:
            regions[0].insert(from_dbu * pya.DBox(via_size))
        elif i < 3:
            for a in range(N):
                for b in range(N):
                    displace = pya.DTrans(edge_gap+a*btwn_via, edge_gap+b*btwn_via) 
                    regions[i].insert(from_dbu * pya.DBox(mcon_side).transformed(displace))
        else:
            for a in range(N):
                displace = pya.DTrans(edge_gap+a*btwn_via, 0) 
                regions[i].insert(from_dbu * pya.DBox(mcon_side, N*(mcon_side+mcon_space)+0.16-mcon_space).transformed(displace))
        
    return regions

metals, layers = gen_via(via_size, via_start, via_end)
to_diff = gen_subvia(via_size)

def place_via(metals, layers, pos):
    metals_dup = []
    for (region, i) in zip(metals, range(len(metals))):
        reg_dup = region.dup()
        reg_dup = reg_dup.transform(pos)
        for polygon in reg_dup:
            Inductor.shapes(layers[i]).insert(to_dbu*polygon)
        
# place_via(metals, layers, pya.CplxTrans(1, 0, 0, 20, 20))
        
#     for polygon in range(len(via)):
#         for i in len(layer_order):
#             for layer_amt in N_layer:
        
            




print(metal_layers)
# layer_num = 0
# for polygon in gen_via(40, 1, 5):
#     for layer in layer_order:
        

[4, 3, 2, 1, 0]


## Via Placement

In [9]:
via_overhang = 0.15

Guard_Inner_Via_Length = Sub_Guard_I_Rad*np.sin(np.pi/8)*2 - via_size

N_vias = int((Guard_Inner_Via_Length-via_size) / (2*via_size))



#im tired and the math just works on this one dont ask me
via_cornerx = -Sub_Guard_I_Rad*np.sin(np.pi/8) + via_size/2 + (Guard_Inner_Via_Length - (N_vias*2*via_size)+2*via_size)/2
via_cornery = Sub_Guard_I_Rad*np.cos(np.pi/8) - via_overhang + via_size/2

via_place_corner = pya.DVector(via_cornerx, via_cornery)





for i in range(8):
    for j in range(N_vias):
        for k in range(2):
            delta_vector = pya.Vector(j*2*via_size, k*2*via_size)
            vias_placement = via_place_corner/dbu + delta_vector/dbu
            t2 = pya.DCplxTrans(1, 45*i, 0, 0, 0)
            t1 = pya.CplxTrans(1, 0, 0, t2*vias_placement)
            place_via(metals, layers, t1)
            if k == 1:
                place_via(to_diff, to_diff_order, t1)
    

In [10]:
# layout.write('3nH_guard.gds')

In [11]:

# up_one = pya.DPoint(0, 1)
# unit_vector = up_one-zero

# for i in range(8):
#     r45 = pya.DCplxTrans(1, 45*, 0, 0.0,0.0)
#     placement =  r * unit_vector 
#     box = pya.DBox(1)
#     box_placement = box.transformed(pya.DTrans(placement))
#     Inductor.shapes(metal1).insert(box_placement)




layout.write('3nH_guard.gds')   